# Ratio Analysis Setup

## Importing Libraries and Data

In [3]:
# Importing Libraries
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set(style = "whitegrid",
        color_codes = True,
        font_scale = 1.5)

The next part assumes one has access to anonymized, confidential participant data from the BEES summer bridge program. If you do not have access, please import data in the format specified in the README and replace import statements to be compatible with your dataset. If you are just interested in the methodology, click the relevant sections below this.

In [4]:
#Importing Data

from google.colab import drive
drive.mount('/content/drive')
pre_2020 = pd.read_excel('/content/drive/My Drive/BEES Research Data/Data for Studies/2020 Pre Survey.xlsx')
pre_2021 = pd.read_excel('/content/drive/My Drive/BEES Research Data/Data for Studies/2021 Pre Survey.xlsx')
pre_2022 = pd.read_excel('/content/drive/My Drive/BEES Research Data/Data for Studies/2022 Pre Survey.xlsx')
pre_2022_new = pd.read_excel('/content/drive/My Drive/BEES Research Data/Data for Studies/2022-BEES-Pre-Survey-for-Shreya.xlsx')
pre_2022 = pd.concat([pre_2022, pre_2022_new.drop(index=(0), columns=["Q1", "Q20"])]).reset_index()
pre_2022=pre_2022.drop("index", axis=1)
pre_2023 = pd.read_excel('/content/drive/My Drive/BEES Research Data/Data for Studies/2023 Pre Survey.xlsx')
post_2020 = pd.read_excel('/content/drive/My Drive/BEES Research Data/Data for Studies/2020 Post Survey.xlsx')
post_2021 = pd.read_excel('/content/drive/My Drive/BEES Research Data/Data for Studies/2021 Post Survey.xlsx')
post_2022 = pd.read_excel('/content/drive/My Drive/BEES Research Data/Data for Studies/2022 Post Survey.xlsx')
post_2023 = pd.read_excel('/content/drive/My Drive/BEES Research Data/Data for Studies/2023 Post Survey.xlsx')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Data Cleaning

In [5]:
# Creating Lists of the dataframes to reduce code redundancy in the future
responses = [pre_2020, pre_2021, pre_2022, pre_2023, post_2020, post_2021, post_2022, post_2023]
unstandardized_column_names = [pre_2020, post_2020, post_2021, post_2022]
pres=[pre_2020, pre_2021, pre_2022, pre_2023]
posts=[post_2020, post_2021, post_2022, post_2023]

# Dropping Intended Majors and other Unnecessary Columns
for response in unstandardized_column_names:
  response.drop(labels= response.columns[[1]], axis=1, inplace=True)
post_2020.drop(labels= post_2020.columns[[1]], axis=1, inplace=True)

# Making column names uniform across all Dataframes
pre_2020.columns = pre_2021.columns
post_2020.columns = post_2021.columns

#Removing ScienceID_5 as the question is "If you are reading this item, please mark "3-Slightly Disagree""
for pre in pres:
  pre.drop(labels= "ScienceID_5", axis=1, inplace=True)
for post in posts:
  post.drop(labels= "ScienceID2_5", axis=1, inplace=True)

#Removing Columns with Qualitative Nominal Responses
pre_post_column_mismatches = ['Q24', 'Q26', 'Q28', 'Q30', 'Q34', 'Q38', 'Q42', 'Q44', 'Q46', 'Q48', 'Q50']
for post in posts:
  post.drop(labels= pre_post_column_mismatches, axis=1, inplace=True)

#Storing Questions for future reference
pre_survey_questions = pre_2022.iloc[0,:].str.replace(r'.*\n\n.* - |.* - ', '', regex=True)[1:]
pre_survey_questions.columns = ['Question_Text']

post_survey_questions = post_2022.iloc[0,:].str.replace(r'.*\n\n.* - |.* - ', '', regex=True)[1:]
post_survey_questions.columns = ['Question_Text']

#Making all the Columns Integers and Resetting Index to ResponseId
pre_2020 = pre_2020[1:].set_index('ResponseId').apply(pd.to_numeric)
post_2020 = post_2020[1:].set_index('ResponseId').apply(pd.to_numeric, errors='ignore')

pre_2021 = pre_2021[1:].set_index('ResponseId').replace(to_replace ='-.*', value = '', regex = True).apply(pd.to_numeric)
post_2021 = post_2021[2:].set_index('ResponseId').replace(to_replace ='-.*', value = '', regex = True).apply(pd.to_numeric, errors='ignore')

pre_2022 = pre_2022[1:].set_index('ResponseId').replace(to_replace ='-.*', value = '', regex = True).apply(pd.to_numeric)
post_2022 = post_2022[1:].set_index('ResponseId').replace(to_replace ='-.*', value = '', regex = True).apply(pd.to_numeric, errors='ignore')

pre_2023 = pre_2023[1:].set_index('ResponseId').replace(to_replace ='-.*', value = '', regex = True).apply(pd.to_numeric)
post_2023 = post_2023[1:].set_index('ResponseId').replace(to_replace ='-.*', value = '', regex = True).apply(pd.to_numeric, errors='ignore')

#Handling Negatively Worded Columns
pre_negative_6_options=["MathSelfConcept_2", "MathSelfConcept_4", "MathSelfConcept_6", "MathSelfConcept_8", "MathSelfConcept_10", "ProgramSelfConcept_2", "ProgramSelfConcept_4", "ProgramSelfConcept_6", "ProgramSelfConcept_8", "ProgramSelfConcept_10", "Mindset_1", "Mindset_2", "Mindset_3"]
pre_negative_4_options=["Concealment_1", "Concealment_2", "Concealment_3", "Concealment_4", "Concealment_5", "Concealment_6", "Concealment_7", "Concealment_8"]
post_negative_6_options=["MathSelfConcept2_2", "MathSelfConcept2_4", "MathSelfConcept2_6", "MathSelfConcept2_8", "MathSelfConcept2_10", "ProgramSelfConcept2_2", "ProgramSelfConcept2_4", "ProgramSelfConcept2_6", "ProgramSelfConcept2_8", "ProgramSelfConcept2_10", "Mindset2_1", "Mindset2_2", "Mindset2_3"]
post_negative_4_options=["Concealment2_1", "Concealment2_2", "Concealment2_3", "Concealment2_4", "Concealment2_5", "Concealment2_6", "Concealment2_7", "Concealment2_8"]

replacement_6_option= {1:6, 2:5, 3:4, 4:3, 5:2, 6:1}
replacement_4_option= {1:4, 2:3, 3:2, 4:1}

for column in pre_negative_6_options:
  pre_2020.replace({column:replacement_6_option}, inplace=True)
  pre_2021.replace({column:replacement_6_option}, inplace=True)
  pre_2022.replace({column:replacement_6_option}, inplace=True)
  pre_2023.replace({column:replacement_6_option}, inplace=True)
for column in pre_negative_4_options:
  pre_2020.replace({column:replacement_4_option}, inplace=True)
  pre_2021.replace({column:replacement_4_option}, inplace=True)
  pre_2022.replace({column:replacement_4_option}, inplace=True)
  pre_2023.replace({column:replacement_4_option}, inplace=True)
for column in post_negative_6_options:
  post_2020.replace({column:replacement_6_option}, inplace=True)
  post_2021.replace({column:replacement_6_option}, inplace=True)
  post_2022.replace({column:replacement_6_option}, inplace=True)
  post_2023.replace({column:replacement_6_option}, inplace=True)
for column in post_negative_4_options:
  post_2020.replace({column:replacement_4_option}, inplace=True)
  post_2021.replace({column:replacement_4_option}, inplace=True)
  post_2022.replace({column:replacement_4_option}, inplace=True)
  post_2023.replace({column:replacement_4_option}, inplace=True)
# Function to select a category of survey questions like Math, Programming, etc.
def select_category(df, cat) :
  return df[[col for col in df.columns if cat in col]]

# Category Labels Dictioanry
cat_labels = {'Math':'Math Self Concept', 'Program':'Program Self Concept', 'ScienceMotivation':'Science Motivation', 'ScienceID':'Science Self Indentity'
              , 'HelpSeeking':'Help Seeking' , 'Concealment':'Concealment', 'Mindset':'Mindset', 'PeerCommunity':'Peer Community'
              , 'CampusBelong':'Campus Belong', 'Resources':'Resources'}

In [8]:
# creating collections for easy access
online_pre_df = pd.concat([pre_2020, pre_2021], ignore_index=True)
online_post_df = pd.concat([post_2020, post_2021], ignore_index=True)
offline_pre_df = pd.concat([pre_2022, pre_2023], ignore_index=True)
offline_post_df = pd.concat([post_2022, post_2023], ignore_index=True)

# to make sure there are uniform indices
offline_intersection = list(set(offline_pre_df.index).intersection(set(offline_post_df.index)))
offline_pre_df = (offline_pre_df.loc[offline_intersection]).sort_index()
offline_post_df = (offline_post_df.loc[offline_intersection]).sort_index()

online_intersection = list(set(online_pre_df.index).intersection(set(online_post_df.index)))
online_pre_df = (online_pre_df.loc[online_intersection]).sort_index()
online_post_df = (online_post_df.loc[online_intersection]).sort_index()

## Ratio Analysis

### Per Question (mentioned in thesis) ###

In [10]:
def per_question_ratio_plots():
  for cat in cat_labels:
    print ("Category: ", cat)
    print()
    offline_pre_df_cat = offline_pre_df.filter(like=cat)
    offline_post_df_cat = offline_post_df.filter(like=cat)
    online_pre_df_cat = online_pre_df.filter(like=cat)
    online_post_df_cat = online_post_df.filter(like=cat)

    max_val = 6
    binwidth_val=0.25
    if cat in ['HelpSeeking', 'Concealment']: # max scores for these categories are 4
      max_val = 4
      binwidth_val=0.20

    for q_num in range(offline_pre_df_cat.shape[1]):

      def scale(s):
        return s.iloc[:,q_num] / max_val

      offline_scaled_pre_score_column = scale(offline_pre_df_cat)
      offline_scaled_post_score_column = scale(offline_post_df_cat)
      offline_post_pre_ratio = offline_scaled_post_score_column / offline_scaled_pre_score_column

      online_scaled_pre_score_column = scale(online_pre_df_cat)
      online_scaled_post_score_column = scale(online_post_df_cat)
      online_post_pre_ratio = online_scaled_post_score_column / online_scaled_pre_score_column

      print(cat ,"Question", q_num)

      sns.histplot(data=offline_post_pre_ratio, stat='density', binwidth=binwidth_val, label='In-person', alpha=0.6) #//for colorless
      sns.histplot(data=online_post_pre_ratio, stat='density', binwidth=binwidth_val, label='Online', alpha = 0.6) #//for colorless
      plt.axvline(1.0, color="red", lw=2) #for black
      plt.title(label = f"{cat} Question {q_num+1}", fontsize = 14)
      plt.xlabel('Post/Pre Ratio', fontsize = 12)
      plt.ylabel('Probability density', fontsize = 12)
      plt.legend(prop={'size': 10})
      plt.show()
      print()

Uncomment and run the cell below to visualize

In [12]:
# per_question_ratio_plots()

### Per Category (not mentioned in thesis) ###

In [13]:
def per_category_ratio_plots():
  for cat in cat_labels:

    offline_pre_df_cat = offline_pre_df.filter(like=cat)
    offline_post_df_cat = offline_post_df.filter(like=cat)
    online_pre_df_cat = online_pre_df.filter(like=cat)
    online_post_df_cat = online_post_df.filter(like=cat)
    max_val = 6
    binwidth_val=0.25
    if cat in ['HelpSeeking', 'Concealment']:
      max_val = 4
      binwidth_val=0.20

    def scale_and_mean(s):
      return (s / max_val).mean(axis=1)

    offline_scaled_mean_pre_score = scale_and_mean(offline_pre_df_cat)
    offline_scaled_mean_post_score = scale_and_mean(offline_post_df_cat)
    offline_post_pre_ratio = offline_scaled_mean_post_score / offline_scaled_mean_pre_score

    online_scaled_mean_pre_score = scale_and_mean(online_pre_df_cat)
    online_scaled_mean_post_score = scale_and_mean(online_post_df_cat)
    online_post_pre_ratio = online_scaled_mean_post_score / online_scaled_mean_pre_score

    sns.histplot(data=offline_post_pre_ratio, stat='density', binwidth=binwidth_val, label='In-person')
    sns.histplot(data=online_post_pre_ratio, stat='density', binwidth=binwidth_val, label='Online')
    plt.axvline(1.0, color="red", lw=2)
    plt.title(cat, fontsize = 14)
    plt.xlabel('Pre - Post Ratio', fontsize = 12)
    plt.ylabel('Probability density', fontsize = 12)
    plt.legend(prop={'size': 10})
    plt.show()
    print()

Uncomment and run the cell below to visualize

In [15]:
#per_category_ratio_plots()